In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
Thu Dec 11 21:45:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8      

In [ ]:
!apt-get update
!apt-get install -y cuda-toolkit-12-4

In [ ]:
import os

os.environ["PATH"] = "/usr/local/cuda-12.4/bin:" + os.environ["PATH"]
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.4/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0


In [ ]:
%%writefile parallel.cu
#include <cuda_runtime.h>
#include <iostream>


__global__ void dkernal (){
    printf("hello world\n");
}

int main(){
    dim3 grid_dim(1);
    dim3 block_dim(32);

    dkernal<<<grid_dim, block_dim>>>();
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess){
        std::cerr << "dkernal launch error: "
                  << cudaGetErrorString(err) << std::endl;
        return -1;
    }
    cudaDeviceSynchronize();


    return 0;
}

Writing parallel.cu


In [ ]:
!nvcc -arch=sm_75 parallel.cu -o parallel
!./parallel

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world


In [ ]:
%%writefile fun.cu
#include <cuda_runtime.h>
#include <iostream>
#define N 128

__global__ void fun(){
    for (int i = 0; i < N; i++){
        printf("%d\t", i * i);
    }
}


int main(){
    fun<<<1, 1>>>();
    cudaDeviceSynchronize();
    return 0;

}

Writing fun.cu


In [ ]:
!nvcc -arch=sm_75 fun.cu -o fun
!./fun

0	1	4	9	16	25	36	49	64	81	100	121	144	169	196	225	256	289	324	361	400	441	484	529	576	625	676	729	784	841	900	961	1024	1089	1156	1225	1296	1369	1444	1521	1600	1681	1764	1849	1936	2025	2116	2209	2304	2401	2500	2601	2704	2809	2916	3025	3136	3249	3364	3481	3600	3721	3844	3969	4096	4225	4356	4489	4624	4761	4900	5041	5184	5329	5476	5625	5776	5929	6084	6241	6400	6561	6724	6889	7056	7225	7396	7569	7744	7921	8100	8281	8464	8649	8836	9025	9216	9409	9604	9801	10000	10201	10404	10609	10816	11025	11236	11449	11664	11881	12100	12321	12544	12769	12996	13225	13456	13689	13924	14161	14400	14641	14884	15129	15376	15625	15876	16129	

In [ ]:
%%writefile fun2.cu
#include <cuda_runtime.h>
#include <iostream>
#define N 128

__global__ void fun(){

  printf("%d\t", threadIdx.x * threadIdx.x);

}


int main(){
    fun<<<1, N>>>();
    cudaDeviceSynchronize();
    return 0;

}

Writing fun2.cu


In [ ]:
!nvcc -arch=sm_75 fun2.cu -o fun2
!./fun2

0	1	4	9	16	25	36	49	64	81	100	121	144	169	196	225	256	289	324	361	400	441	484	529	576	625	676	729	784	841	900	961	1024	1089	1156	1225	1296	1369	1444	1521	1600	1681	1764	1849	1936	2025	2116	2209	2304	2401	2500	2601	2704	2809	2916	3025	3136	3249	3364	3481	3600	3721	3844	3969	4096	4225	4356	4489	4624	4761	4900	5041	5184	5329	5476	5625	5776	5929	6084	6241	6400	6561	6724	6889	7056	7225	7396	7569	7744	7921	8100	8281	8464	8649	8836	9025	9216	9409	9604	9801	10000	10201	10404	10609	10816	11025	11236	11449	11664	11881	12100	12321	12544	12769	12996	13225	13456	13689	13924	14161	14400	14641	14884	15129	15376	15625	15876	16129	

In [ ]:
!nvprof ./fun2

==4153== NVPROF is profiling process 4153, command: ./fun2
1024	1089	1156	1225	1296	1369	1444	1521	1600	1681	1764	1849	1936	2025	2116	2209	2304	2401	2500	2601	2704	2809	2916	3025	3136	3249	3364	3481	3600	3721	3844	3969	4096	4225	4356	4489	4624	4761	4900	5041	5184	5329	5476	5625	5776	5929	6084	6241	6400	6561	6724	6889	7056	7225	7396	7569	7744	7921	8100	8281	8464	8649	8836	9025	9216	9409	9604	9801	10000	10201	10404	10609	10816	11025	11236	11449	11664	11881	12100	12321	12544	12769	12996	13225	13456	13689	13924	14161	14400	14641	14884	15129	15376	15625	15876	16129	0	1	4	9	16	25	36	49	64	81	100	121	144	169	196	225	256	289	324	361	400	441	484	529	576	625	676	729	784	841	900	961	==4153== Profiling application: ./fun2
==4153== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  227.39us         1  227.39us  227.39us  227.39us  fun(void)
      API calls:   99.36%  70.832ms         1  70.832ms  70.832ms  70.832ms  cudaLau

In [ ]:
!nvprof ./fun

==4176== NVPROF is profiling process 4176, command: ./fun
0	1	4	9	16	25	36	49	64	81	100	121	144	169	196	225	256	289	324	361	400	441	484	529	576	625	676	729	784	841	900	961	1024	1089	1156	1225	1296	1369	1444	1521	1600	1681	1764	1849	1936	2025	2116	2209	2304	2401	2500	2601	2704	2809	2916	3025	3136	3249	3364	3481	3600	3721	3844	3969	4096	4225	4356	4489	4624	4761	4900	5041	5184	5329	5476	5625	5776	5929	6084	6241	6400	6561	6724	6889	7056	7225	7396	7569	7744	7921	8100	8281	8464	8649	8836	9025	9216	9409	9604	9801	10000	10201	10404	10609	10816	11025	11236	11449	11664	11881	12100	12321	12544	12769	12996	13225	13456	13689	13924	14161	14400	14641	14884	15129	15376	15625	15876	16129	==4176== Profiling application: ./fun
==4176== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  6.7720ms         1  6.7720ms  6.7720ms  6.7720ms  fun(void)
      API calls:   93.17%  94.900ms         1  94.900ms  94.900ms  94.900ms  cudaLaunc

In [ ]:
%%writefile malloc.cu
#include <iostream>
#include <cuda_runtime.h>
#define N 128

__global__ void square(int *a){
    a[threadIdx.x] = threadIdx.x * threadIdx.x;
}

int main(){
    int a[N], *da;
    int NBtye = N * sizeof(int);


    cudaMallocManaged((void**)&da, NBtye);
    dim3 grid_dim(1);
    dim3 block_dim(N);
    square<<<grid_dim, block_dim>>>(da);
    cudaMemcpy(a, da, NBtye, cudaMemcpyDeviceToHost);
    for (int i = 0; i < N; i++){
        printf("%d\t", a[i]);
    }
    return 0;
}

Writing malloc.cu


In [ ]:
!nvcc -arch=sm_75 malloc.cu -o malloc
!./malloc

0	1	4	9	16	25	36	49	64	81	100	121	144	169	196	225	256	289	324	361	400	441	484	529	576	625	676	729	784	841	900	961	1024	1089	1156	1225	1296	1369	1444	1521	1600	1681	1764	1849	1936	2025	2116	2209	2304	2401	2500	2601	2704	2809	2916	3025	3136	3249	3364	3481	3600	3721	3844	3969	4096	4225	4356	4489	4624	4761	4900	5041	5184	5329	5476	5625	5776	5929	6084	6241	6400	6561	6724	6889	7056	7225	7396	7569	7744	7921	8100	8281	8464	8649	8836	9025	9216	9409	9604	9801	10000	10201	10404	10609	10816	11025	11236	11449	11664	11881	12100	12321	12544	12769	12996	13225	13456	13689	13924	14161	14400	14641	14884	15129	15376	15625	15876	16129	

In [ ]:
%%writefile array_space.cu

#include <iostream>
#include <cuda_runtime.h>

// Simple CUDA error-checking macro
#define CUDA_CHECK(call)                                              \
    do {                                                              \
        cudaError_t err = call;                                       \
        if (err != cudaSuccess) {                                     \
            std::cerr << "CUDA error: " << cudaGetErrorString(err)    \
                      << " at " << __FILE__ << ":" << __LINE__        \
                      << std::endl;                                   \
            std::exit(EXIT_FAILURE);                                  \
        }                                                             \
    } while (0)

// Kernel 1: initialize all elements to zero in parallel
__global__ void init_zero(int *a, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;  // global index
    if (idx < N) {
        a[idx] = 0;
    }
}

// Kernel 2: perform a[i] += i in parallel
__global__ void add_index(int *a, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        a[idx] += idx;
    }
}

// Function to run one test case:
// (1) initialize array to zero
// (2) add index to each element (a[i] += i)
// (3) verify that a[i] == i
void run_case(int N) {
    std::cout << "\n=== Testing N = " << N << " ===\n";

    int *d_a = nullptr;
    size_t bytes = N * sizeof(int);

    // Allocate unified memory for convenience
    CUDA_CHECK(cudaMallocManaged(&d_a, bytes));

    // Configure kernel launch: 256 threads per block
    int threads = 256;
    int blocks  = (N + threads - 1) / threads;

    dim3 block_dim(threads);
    dim3 grid_dim(blocks);

    // 1. Initialize array to zero
    init_zero<<<grid_dim, block_dim>>>(d_a, N);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    // 2. Perform a[i] += i
    add_index<<<grid_dim, block_dim>>>(d_a, N);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    // 3. Verify correctness
    bool ok = true;
    for (int i = 0; i < N; ++i) {
        if (d_a[i] != i) {
            std::cout << "Mismatch at i=" << i
                      << ", got " << d_a[i]
                      << ", expected " << i << std::endl;
            ok = false;
            break;
        }
    }

    if (ok) {
        std::cout << "Result correct for N = " << N << " ✔" << std::endl;
    } else {
        std::cout << "Result incorrect for N = " << N << " ✘" << std::endl;
    }

    // Print first few elements for visual confirmation
    int to_print = (N < 10 ? N : 10);
    std::cout << "First " << to_print << " elements: ";
    for (int i = 0; i < to_print; ++i) {
        std::cout << d_a[i] << " ";
    }
    std::cout << std::endl;

    CUDA_CHECK(cudaFree(d_a));
}

int main() {
    // Run tests for the three required array sizes
    run_case(32);     // Task 1
    run_case(1024);   // Task 2
    run_case(8000);   // Task 4 & 5

    return 0;
}

Writing array_space.cu


In [ ]:
!nvcc -arch=sm_75 array_space.cu -o array_space
!./array_space


=== Testing N = 32 ===
Result correct for N = 32 ✔
First 10 elements: 0 1 2 3 4 5 6 7 8 9 

=== Testing N = 1024 ===
Result correct for N = 1024 ✔
First 10 elements: 0 1 2 3 4 5 6 7 8 9 

=== Testing N = 8000 ===
Result correct for N = 8000 ✔
First 10 elements: 0 1 2 3 4 5 6 7 8 9 


In [ ]:
%%writefile home_work.cu
#include <iostream>
#include <cuda_runtime.h>

// Simple CUDA error-checking macro
#define CUDA_CHECK(call)                                              \
    do {                                                              \
        cudaError_t err = call;                                       \
        if (err != cudaSuccess) {                                     \
            std::cerr << "CUDA error: " << cudaGetErrorString(err)    \
                      << " at " << __FILE__ << ":" << __LINE__        \
                      << std::endl;                                   \
            std::exit(EXIT_FAILURE);                                  \
        }                                                             \
    } while (0)

__global__ void square_cubic_sum(int *device_x, int *device_y, int *device_z, int N){
    int idx = blockIdx.x * blockDim.x + threadIdx.x;  // global index
    if (idx < N){device_z[idx] = device_x[idx] * device_x[idx] + device_y[idx] * device_y[idx] * device_y[idx];}


}


int main(){

    int host_x[] = {1, 3, 5};
    int host_y[] = {2, 4, 6};
    int host_z[3];

    int *device_x = nullptr;
    int *device_y = nullptr;
    int *device_z = nullptr;

    int N = sizeof(host_x) / sizeof(host_x[0]);

    size_t bytes = N * sizeof(int);

    // Allocate unified memory for convenience
    CUDA_CHECK(cudaMallocManaged(&device_x, bytes));
    CUDA_CHECK(cudaMallocManaged(&device_y, bytes));
    CUDA_CHECK(cudaMallocManaged(&device_z, bytes));

    CUDA_CHECK(cudaMemcpy(device_x, host_x, bytes, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(device_y, host_y, bytes, cudaMemcpyHostToDevice));


    // Configure kernel launch: 256 threads per block
    int threads = 256;
    int blocks  = (N + threads - 1) / threads;

    dim3 block_dim(threads);
    dim3 grid_dim(blocks);

    square_cubic_sum<<<grid_dim, block_dim>>>(device_x, device_y, device_z, N);
    cudaMemcpy(host_z, device_z, bytes, cudaMemcpyDeviceToHost);

    for(int i = 0; i < N; i++){
        std::cout<< host_z[i] << " ";
    }

    std::cout<< std::endl;

}

Writing home_work.cu


In [ ]:
!nvcc -arch=sm_75 home_work.cu -o home_work
!./home_work

9 73 241 


In [ ]:
%%writefile mem_syn.cu
#include <iostream>
#include <cuda_runtime.h>
#include <math.h>

// Simple CUDA error-checking macro
#define CUDA_CHECK(call)                                              \
    do {                                                              \
        cudaError_t err = call;                                       \
        if (err != cudaSuccess) {                                     \
            std::cerr << "CUDA error: " << cudaGetErrorString(err)    \
                      << " at " << __FILE__ << ":" << __LINE__        \
                      << std::endl;                                   \
            std::exit(EXIT_FAILURE);                                  \
        }                                                             \
    } while (0)

__global__ void kernel(int* x, int* y, int* z, int N){
    int id = blockIdx.x * blockDim.x + threadIdx.x;  // global index
    if (id < N){
        int vx = x[id];
        int vy = y[id];
        z[id] = vx * vx + vy * vy *vy;
    }
}

int main(){
    //int x[3] = {1, 3, 5};
    //int y[3] = {2, 4, 6};
    //int z[3];


    int *x = nullptr;
    int *y = nullptr;
    int *z = nullptr;


    int N = 3;
    size_t bytes = N * sizeof(int);

    CUDA_CHECK(cudaMallocManaged(&x, bytes));
    CUDA_CHECK(cudaMallocManaged(&y, bytes));
    CUDA_CHECK(cudaMallocManaged(&z, bytes));

    x[0]=1; x[1]=3; x[2]=5;
    y[0]=2; y[1]=4; y[2]=6;

    int threads = 256;
    int blocks  = (N + threads - 1) / threads;
    dim3 block_dim(threads);
    dim3 grid_dim(blocks);

    kernel<<<grid_dim, block_dim>>>(x, y, z, N);

    CUDA_CHECK(cudaDeviceSynchronize());

    for(int i = 0; i < N; i++){
        std::cout<< z[i] << " ";
    }

    std::cout<< std::endl;


    return 0;
}


Writing mem_syn.cu


In [ ]:
!nvcc -arch=sm_75 mem_syn.cu -o mem_syn
!./mem_syn

9 73 241 


In [ ]:
%%writefile Non_share_mem.cu
#include <iostream>
#include <cuda_runtime.h>
#define N 1024


__global__ void matrix_add(const int* M_in, int* M_out){
    int row = blockIdx.x;     // 0..N-1
    int col = threadIdx.x;    // 0..N-1

    int idx = row * N + col;

    if (col < N - 1){
        M_out[idx] =  M_in[idx] + M_in[idx + 1];
    }else{
        M_out[idx] = M_in[idx];
    }

}


int main(){

    size_t bytes = N * N * sizeof(int);


    //host variables
    int* h_M = (int*)malloc(bytes);
    int* h_M_out = (int*)malloc(bytes);
    for(int i = 0; i < N * N; i++){
        h_M[i] = i * i;
    }


    // device variables

    int *d_M_in, *d_M_out;
    cudaMalloc(&d_M_in, bytes);
    cudaMalloc(&d_M_out, bytes);

     cudaMemcpy(d_M_in, h_M, bytes, cudaMemcpyHostToDevice);

     dim3 grid_dim(N);
     dim3 block_dim(N);

     matrix_add<<<grid_dim, block_dim>>>(d_M_in, d_M_out);
     cudaDeviceSynchronize();

     cudaMemcpy(h_M_out, d_M_out, bytes, cudaMemcpyDeviceToHost);

     for (int j = 0; j < 8; ++j)
        std::cout << h_M[j] << " ";
     std::cout << std::endl;

    cudaFree(d_M_in);
    cudaFree(d_M_out);
    free(h_M);

    return 0;
}



Overwriting share_mem.cu


In [ ]:
!nvcc -arch=sm_75 Non_share_mem.cu -o Non_share_mem
!./Non_share_mem

0 1 4 9 16 25 36 49 


In [ ]:
%%writefile share_mem.cu
#include <iostream>
#include <cuda_runtime.h>
#define N 1024


__global__ void matrix_add(int* M){
    int row = blockIdx.x;     // 0..N-1
    int col = threadIdx.x;    // 0..N-1

    int idx = row * N + col;

    extern __shared__ int srow[];
    srow[col] = M[idx];
     __syncthreads();  // 确保整行都已写入 srow



    if (col < N - 1){
        M[idx] =  srow[col] + srow[col + 1];
    }else{
        M[idx] = srow[col];
    }

}


int main(){

    size_t bytes = N * N * sizeof(int);


    //host variables
    int* h_M = (int*)malloc(bytes);
    for(int i = 0; i < N * N; i++){
        h_M[i] = i * i;
    }


    // device variables

     int *d_M;
     cudaMalloc(&d_M, bytes);

     cudaMemcpy(d_M, h_M, bytes, cudaMemcpyHostToDevice);

     dim3 grid_dim(N);
     dim3 block_dim(N);

     size_t shmemBytes = N * sizeof(int);

     matrix_add<<<grid_dim, block_dim, shmemBytes>>>(d_M);
     cudaDeviceSynchronize();

     cudaMemcpy(h_M, d_M, bytes, cudaMemcpyDeviceToHost);

     for (int j = 0; j < 8; ++j)
        std::cout << h_M[j] << " ";
     std::cout << std::endl;

    cudaFree(d_M);
    free(h_M);

    return 0;
}



Overwriting share_mem.cu


In [ ]:
!nvcc -arch=sm_75 share_mem.cu -o share_mem
!./share_mem

1 5 13 25 41 61 85 113 


In [ ]:
%%writefile unifying_share_mem.cu

#include <iostream>
#include <cuda_runtime.h>
#define N 1024


__global__ void matrix_add(int* M){
    int row = blockIdx.x;     // 0..N-1
    int col = threadIdx.x;    // 0..N-1

    int idx = row * N + col;

    extern __shared__ int srow[];
    srow[col] = M[idx];
     __syncthreads();  // 确保整行都已写入 srow



    if (col < N - 1){
        M[idx] =  srow[col] + srow[col + 1];
    }else{
        M[idx] = srow[col];
    }

}


int main(){

    size_t bytes = N * N * sizeof(int);

    int *M;
    //unified memory
    cudaMallocManaged(&M, bytes);

    for(int i = 0; i < N * N; ++i){
        M[i] = i * i;
    }



     dim3 grid_dim(N);
     dim3 block_dim(N);

     size_t shmemBytes = N * sizeof(int);

     matrix_add<<<grid_dim, block_dim, shmemBytes>>>(M);
     cudaDeviceSynchronize();

     // cudaMemcpy(h_M, d_M, bytes, cudaMemcpyDeviceToHost);

     for (int j = 0; j < 8; ++j)
        std::cout << M[j] << " ";
     std::cout << std::endl;

    cudaFree(M);


    return 0;
}



Overwriting unifying_share_mem.cu


In [ ]:
!nvcc -arch=sm_75 unifying_share_mem.cu -o unifying_share_mem
!./unifying_share_mem

1 5 13 25 41 61 85 113 
